In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options  

#import sqlite
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getHTML(path, hidden=True):

    if hidden:
        chr_options = Options()  
        chr_options.add_argument("--headless")        
        driver = webdriver.Chrome(options = chr_options)
    else:
        driver = webdriver.Chrome()
        
    driver.get(path)
    pageHTML = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    return pageHTML

In [ ]:
web_page = 'https://www.cv.lv/darba-sludinajumi/visi'
web = getHTML(web_page)

web = web.find_all('div', {'class':'form-group'})
categories = []

for w in web:
    if w.find('input', {'name':'filters[]'}) != None:
        if w.find('input').get('id')[:8] == 'cb-Tegvk':
            if len(w.text) > 2:
                link = 'https://www.cv.lv' + w.find('a').get('href')
                categories.append([str(w.text).split(' (')[0].strip(), link])

joblist = []

for cat in categories:

    next_page = cat[1]

    while next_page != None:
        cvlv = getHTML(next_page)
        jobs = cvlv.find('div', id='joblist').find_all('div', {'class':'offer_primary'})

        for o in jobs:
            title = o.find('h2').text
            company = o.find('span', {'itemprop':'hiringOrganization'}).text
            category = cat[0]
            salary = o.find('span', {'class':'salary-blue'})
            place = o.find('span', {'itemprop':'jobLocation'}).text
            link = o.find('h2').find('a').get('href')
            date = o.find('span', {'itemprop':'datePosted'}).get('content')

            if o.find('span', {'itemprop':'jobLocation'}).find('a') != None:
                coordinates = o.find('span', {'itemprop':'jobLocation'}).find('a').get('href')
                coordinates = coordinates.split('=')[1].split('%3A')
                try:
                    coordinates[0] = float(coordinates[0])
                    coordinates[1] = float(coordinates[1])
                except ValueError:
                    coordinates = 'Not found'
            else:
                coordinates = 'Not found'

            if salary != None:
                joblist.append([title,company,category,link,place,coordinates,date,salary.text])

        if cvlv.find('li', {'class':'page_next'}) == None:
            break
        else:
            if cvlv.find('li', {'class':'page_next'}).find('a') == None:
                break
            else:
                next_page = cvlv.find('li', {'class':'page_next'}).find('a').get('href')
                next_page = 'https://www.cv.lv' + str(next_page)

    print('{} processed'.format(category))

job_list = pd.DataFrame(joblist, columns = list(['Title', 'Company', 'Category', 'Link',
                                                     'Place', 'Coordinates', 
                                                     'Date', 'Salary']))
joblist = []
job_list.to_csv('Y:\\job_salaries.csv', encoding='utf-16')

Prakse processed
Administratīvais darbs / Asistēšana processed
Bankas / Apdrošināšana processed
Brīvprātīgo darbs processed
Būvniecība / Nekustamais īpašums processed
Cilvēkresursi processed
Drošība / Glābšanas dienesti / Aizsardzība processed
Elektronika / Telekomunikācijas processed
Enerģētika / Elektroenerģija processed
Farmācija processed
Finanses / Grāmatvedība processed
Informācijas tehnoloģijas processed
Izglītība / Zinātne processed
Jurisprudence / Tieslietas processed
Kultūra / Māksla / Izklaide / Sports processed
Kvalitātes vadība / Kvalitātes kontrole processed
Lauksaimniecība / Vides zinātne processed
Medicīna / Sociālā aprūpe processed
Mediji / Sabiedriskās attiecības processed
Mežsaimniecība / Kokapstrāde processed
Mārketings / Reklāma processed
Pakalpojumi processed


In [ ]:
job_list['Pay period'] = ''
job_list['Monthly Salary'] = 0.0
job_list['Starting Salary'] = 0.0
job_list['Max Salary'] = 0.0

for i in range(len(job_list)):
    if str(job_list.loc[i, 'Salary']).find('Alga mēnesī: ') > -1:
        job_list.loc[i,'Pay period'] = 'MONTHLY'
        salary = str(job_list.loc[i, 'Salary'])
        
        if salary.find('līdz') > -1:
            if len(str(salary.split(': ')[1].split('līdz')[0]).strip()) > 3:
                minSalary = float(str(salary.split(': ')[1].split('līdz')[0]).strip())
                maxSalary = float(str(salary.split(': ')[1].\
                                      split('līdz')[1].split(' ')[1]).strip())
                salary = (minSalary + maxSalary) / 2
                
                job_list.loc[i,'Monthly Salary'] = salary
                job_list.loc[i,'Starting Salary'] = minSalary
                job_list.loc[i,'Max Salary'] = maxSalary
            else:
                minSalary = 0.0
                maxSalary = 0.0
                salary = float(str(salary.split(': ')[1].split('līdz')[1].split(' ')[1])\
                               .strip())
                job_list.loc[i,'Monthly Salary'] = salary * 8 * 22 #22 working days
        else:
            salary = float(salary.split(': ')[1].split(' ')[0])
            job_list.loc[i,'Monthly Salary'] = salary
            
    elif str(job_list.loc[i, 'Salary']).find('Stundas likme: ') > -1:
        job_list.loc[i, 'Pay period'] = 'HOURLY'
        salary = str(job_list.loc[i, 'Salary'])
                
        if salary.find('līdz') > -1:
            if len(str(salary.split(': ')[1].split('līdz')[0]).strip()) > 3:
                minSalary = float(str(salary.split(': ')[1].split('līdz')[0]).strip())
                maxSalary = float(str(salary.split(': ')[1].\
                                      split('līdz')[1].split(' ')[1]).strip())
                salary = (minSalary + maxSalary) / 2
                
                job_list.loc[i,'Monthly Salary'] = salary * 8 * 22 #22 working days
                job_list.loc[i,'Starting Salary'] = minSalary * 8 * 22 #22 working days
                job_list.loc[i,'Max Salary'] = maxSalary * 8 * 22 #22 working days
            else:
                minSalary = 0.0
                maxSalary = 0.0
                salary = float(str(salary.split(': ')[1].split('līdz')[1].split(' ')[1])\
                               .strip()) * 8 * 22 #22 working days
                job_list.loc[i,'Monthly Salary'] = salary * 8 * 22 #22 working days
        else:
            salary = float(salary.split(': ')[1].split(' ')[0])
            job_list.loc[i,'Monthly Salary'] = salary * 8 * 22 #22 working days
    else:
        print(job_list.loc[i])

job_list.to_csv('Y:\\Cleaner_job_salaries.csv', encoding='utf-8')
avg_pay = job_list.groupby('Pay period')[['Monthly Salary']].quantile(0.5)
avg_pay['Min Pay'] = job_list.groupby('Pay period')['Monthly Salary'].quantile(0.01)
avg_pay['Max Pay'] = job_list.groupby('Pay period')['Monthly Salary'].quantile(0.97)
avg_pay

In [ ]:
for i in range(len(job_list)):
    if ((job_list.loc[i,'Monthly Salary'] < avg_pay['Min Pay'].min())
    | (job_list.loc[i,'Monthly Salary'] > avg_pay['Max Pay'].max())):
        
        sal = job_list.loc[i,'Monthly Salary']
        if (((sal / (22 * 8)) <= avg_pay['Max Pay'].mean())
        & ((sal / (22 * 8)) >= avg_pay['Min Pay'].mean())):
            job_list.loc[i,'Monthly Salary'] = sal / (22 * 8)
            
        elif (((sal * (22 * 8)) <= avg_pay['Max Pay'].mean())
        & ((sal * (22 * 8)) >= avg_pay['Min Pay'].mean())):
            job_list.loc[i,'Monthly Salary'] = sal * (22 * 8)
            
        elif (((sal / (22 * 8 * 12)) <= avg_pay['Max Pay'].mean())
        & ((sal / (22 * 8 * 12)) >= avg_pay['Min Pay'].mean())):
            job_list.loc[i,'Monthly Salary'] = sal/(22 * 8 * 12)
        
        elif (((sal / 12) <= avg_pay['Max Pay'].mean())
        & ((sal / 12) >= avg_pay['Min Pay'].mean())):
            job_list.loc[i,'Monthly Salary'] = sal/12
            
        else:
            sal = job_list.loc[i,'Monthly Salary']
            annual = sal / (22 * 8)
            hourly = sal * 22 * 8
            
            if str(job_list.loc[i,'Salary']).find(str(annual)) > -1:
                job_list.loc[i,'Monthly Salary'] = annual
            
            elif str(job_list.loc[i,'Salary']).find(str(hourly)) > -1:
                job_list.loc[i,'Monthly Salary'] = hourly
                
            elif str(job_list.loc[i,'Salary']).find(str(int(sal))) > -1:
                job_list.loc[i,'Monthly Salary'] = sal
            
job_list.to_csv('Y:\\Clean_job_salaries.csv', encoding='utf-16')
job_list.describe()

In [ ]:
companies = job_list.groupby('Company')[['Monthly Salary', 'Starting Salary', 
                                         'Max Salary']].mean()

jobs_comp = job_list.groupby('Company')[['Monthly Salary']].count()
popular_comp = jobs_comp.where(jobs_comp['Monthly Salary'] > 10).dropna()

popular_comp['Salary'] = companies['Monthly Salary']
popular_comp.where(popular_comp['Salary'] > 0).dropna()['Salary'].plot(kind='barh')

positions = job_list.groupby('Title')[['Monthly Salary', 
                                         'Starting Salary', 'Max Salary']].mean()

popular_positions = job_list.groupby('Title')[['Monthly Salary']].count()
popular_positions = popular_positions.where(popular_positions['Monthly Salary']>2).dropna()
popular_positions['Salary'] = positions['Monthly Salary']
popular_positions.sort_values(by='Salary', ascending=False)

In [ ]:
top_payers = popular_comp.where(
    (popular_comp['Salary'] > popular_comp['Salary'].quantile(0.75)) &
    (popular_comp['Monthly Salary'] >= popular_comp['Monthly Salary'].quantile(0.3)))
top_payers.dropna(inplace=True)
#top_payers['Positions'] = 
job_list.where((job_list['Company'].isin(top_payers.index)) &
              (job_list['Monthly Salary'] > job_list['Monthly Salary'].quantile(0.9))
              ).dropna()[['Company','Title','Monthly Salary']].sort_values(by='Company')

In [ ]:
job_list['Date'] = pd.to_datetime(job_list['Date'])
job_list.groupby('Date')['Monthly Salary'].quantile(0.5)\
.rolling(20).mean().dropna().plot(figsize=(18,6))